<a href="https://colab.research.google.com/github/saadahmedsh/sarcasm_detection_BERT/blob/main/sarcasm_detection_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from google.colab import files

In [ ]:
file=files.upload()

Saving Sarcasm Dataset.csv to Sarcasm Dataset.csv


In [ ]:
df=pd.read_csv("Sarcasm Dataset.csv")

In [ ]:
df.head()

,Unnamed: 0,tweet,sarcastic,sarcasm,irony,satire,understatement,overstatement,rhetorical_question
0,0,The only thing I got from college is a caffein...,1,0.0,1.0,0.0,0.0,0.0,0.0
1,1,I love it when professors draw a big question ...,1,1.0,0.0,0.0,0.0,0.0,0.0
2,2,Remember the hundred emails from companies whe...,1,0.0,1.0,0.0,0.0,0.0,0.0
3,3,Today my pop-pop told me I was not “forced” to...,1,1.0,0.0,0.0,0.0,0.0,0.0
4,4,@VolphanCarol @littlewhitty @mysticalmanatee I...,1,1.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df['tweet'],df['sarcastic'], stratify=df['sarcastic'])


In [ ]:
!pip install "tensorflow-text==2.8.*"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.9 MB 4.7 MB/s 


In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as tf_text

In [ ]:
#importing BERT Layers

preprocessing_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
encoding_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [ ]:
def trainBERT():
  text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
  preprocessed_text = preprocessing_layer(text_input)
  outputs = encoding_layer(preprocessed_text)

  layers = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
  layers = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(layers)
  model = tf.keras.Model(inputs=[text_input], outputs = [layers])

  METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
  ]

  model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=METRICS)
  x = np.asarray(X_train).astype('str')
  model.fit(x, y_train, epochs=1)
  # y_predicted = model.predict(X_test)
  # return y_predicted
  return model





In [ ]:
trained_BERT=trainBERT()

82/82 [==============================] - 38s 347ms/step - loss: 0.5892 - accuracy: 0.7370 - precision: 0.0952 - recall: 0.0062


In [ ]:
x = np.asarray(X_test).astype('str')

In [ ]:
y_pred=trained_BERT.predict(x)

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
y_pred = np.where(y_pred > 0.5, 1, 0)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.75      1.00      0.86       650
           1       0.00      0.00      0.00       217

    accuracy                           0.75       867
   macro avg       0.37      0.50      0.43       867
weighted avg       0.56      0.75      0.64       867



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Taken from kaggle

import tensorflow as tf
from transformers import BertTokenizer, TFBertModel

def getBERTEmbeddings(modelName,tokenizer,name,input_fields):
    tokenizer = tokenizer.from_pretrained(name)
    model = modelName.from_pretrained(name)
    input_ids = tf.constant(tokenizer.encode(input_fields))[None, :]  # Batch size 1
    outputs = model(input_ids)
    last_hidden_states = outputs[0]
    tokens=last_hidden_states[0]
    return tokens


In [ ]:
tokens=getBERTEmbeddings(TFBertModel,BertTokenizer,'bert-base-uncased',df['tweet'].to_list())

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.
Token indices sequence length is longer than the specified maximum sequence length for this model (3470 > 512). Running this sequence through the model will result

In [ ]:
tweets=np.array(tokens)

In [ ]:
# train test for embeddings from BERT
X_train, X_test, y_train, y_test = train_test_split(tweets[0:len(df)],df['sarcastic'], stratify=df['sarcastic'])

In [ ]:
from tensorflow import keras
from tensorflow.keras import Sequential, layers
from tensorflow.keras.layers import LSTM
from keras.layers import Dense, Embedding, SpatialDropout1D

In [ ]:
def trainLSTM():
  model = Sequential()
  model.add(Embedding(tokens.shape[0], 58,input_length = tokens.shape[1]))
  model.add(SpatialDropout1D(0.4))
  model.add(LSTM(196, dropout=0.3, recurrent_dropout=0.3, return_sequences=True))
  model.add(LSTM(196, dropout=0.3, recurrent_dropout=0.3))
  model.add(Dense(1,activation='softmax'))
  model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
  lstm_train = model.fit(X_train, y_train, epochs = 1, batch_size=32, verbose = 1)
  values=model.predict(X_test)
  return values

In [ ]:
# Creating LSTM model
y_predicted_bert_embeddings=trainLSTM()

82/82 [==============================] - 518s 6s/step - loss: 0.0000e+00 - accuracy: 0.2499


In [ ]:
y_predicted_bert_embeddings = np.where(y_predicted_bert_embeddings > 0.5, 1, 0)
print(classification_report(y_test, y_predicted_bert_embeddings))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       650
           1       0.25      1.00      0.40       217

    accuracy                           0.25       867
   macro avg       0.13      0.50      0.20       867
weighted avg       0.06      0.25      0.10       867



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
